# Import requirements

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 14.9 MB/s 
     |████████████████████████████████| 596 kB 77.7 MB/s 
     |████████████████████████████████| 895 kB 73.1 MB/s 
     |████████████████████████████████| 6.5 MB 63.7 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 16.1 MB/s 
     |████████████████████████████████| 144 kB 70.5 MB/s 
     |████████████████████████████████| 181 kB 71.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4dee91b8b46254d4d3813b5fa9d844d0587cdd9c602ec7fc99c548a24f2522ab
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)
import math
import wandb

# 1. Preprocess

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/goorm-nlp-projects-3rd/'

Mounted at /content/drive


In [ ]:
def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join(file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    train_pos = make_data_strings(drive_path + 'sentiment.train.1')
    train_neg = make_data_strings(drive_path + 'sentiment.train.0')
    dev_pos = make_data_strings(drive_path + 'sentiment.dev.1')
    dev_neg = make_data_strings(drive_path + 'sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
!ls

drive  sample_data


In [1]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)  # 최초에만 실행, csv파일 저장 후에는 실행하지 않아도 됨

In [ ]:
train_pos[:10]

['101 6581 2833 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 2204 15174 2098 7570 22974 2063 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 2204 2326 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 1996 2047 2846 3504 6429 1012 102']

In [ ]:
import pandas as pd
def data_to_csv(file_name, data):
    df_data = pd.DataFrame(data)
    df_data.to_csv(file_name + '.csv', index=False)

def csv_to_data(file_name):
    tmp_df = pd.read_csv(file_name)
    tmp = tmp_df.values
    data_strings = []
    for item in tmp:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [ ]:
# 토큰화된 데이터를 csv로 저장하는 셀
data_to_csv(drive_path + 'train_pos', train_pos)
data_to_csv(drive_path + 'train_neg', train_neg)
data_to_csv(drive_path + 'dev_pos', dev_pos)
data_to_csv(drive_path + 'dev_neg', dev_neg)

In [ ]:
# 토큰데이터 csv파일을 불러오는 셀
train_pos = csv_to_data(drive_path + 'train_pos.csv')
train_neg = csv_to_data(drive_path + 'train_neg.csv')
dev_pos = csv_to_data(drive_path + 'dev_pos.csv')
dev_neg = csv_to_data(drive_path + 'dev_neg.csv')

In [ ]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [ ]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [ ]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

(array([ 101, 6581, 2833, 1012,  102]), array([1]))
(array([  101, 21688,  8013,  2326,  1012,   102]), array([1]))
(array([  101,  2027,  2036,  2031,  3679, 19247,  1998,  3256,  6949,
        2029,  2003,  2428,  2204,  1012,   102]), array([1]))
(array([  101,  2009,  1005,  1055,  1037,  2204, 15174,  2098,  7570,
       22974,  2063,  1012,   102]), array([1]))
(array([ 101, 1996, 3095, 2003, 5379, 1012,  102]), array([1]))
(array([ 101, 2204, 3347, 2833, 1012,  102]), array([1]))
(array([ 101, 2204, 2326, 1012,  102]), array([1]))
(array([  101, 11350,  1997,  2154,  2003, 25628,  1998,  7167,  1997,
       19247,  1012,   102]), array([1]))
(array([  101,  2307,  2173,  2005,  6265,  2030,  3347, 27962,  1998,
        5404,  1012,   102]), array([1]))
(array([ 101, 1996, 2047, 2846, 3504, 6429, 1012,  102]), array([1]))
(array([ 101, 2023, 2173, 2001, 2200, 2204, 1012,  102]), array([1]))


In [ ]:
# 데이터를 섞어주는 함수
def switch_data(t_pos, t_neg, v_pos, v_neg, k, n):
    # k = total epochs
    # n = epoch for now
    if n > k-1:  # epoch값이 최대치보다 크게 들어가면 데이터셋이 바뀌는 오류... 
        raise ValueError("larger epoch than the largest epoch can't be an input value")
    if n > 0: # 첫 epoch 에선 switching 하지 않음
        gap_pos = len(v_pos)//(k-1) # switching range
        gap_neg = len(v_pos)//(k-1)
        start_pos = (n-1)*gap_pos
        start_neg = (n-1)*gap_neg
        # pos dataset switch         
        tmp = v_pos[start_pos:start_pos+gap_pos]        
        v_pos[start_pos:start_pos+gap_pos] = t_pos[start_pos:start_pos+gap_pos]
        t_pos[start_pos:start_pos+gap_pos] = tmp
        # neg dataset switch
        tmp = v_neg[start_neg:start_neg+gap_neg]
        v_neg[start_neg:start_neg+gap_neg] = t_neg[start_neg:start_neg+gap_neg]
        t_neg[start_neg:start_neg+gap_neg] = tmp
        
    return t_pos, t_neg, v_pos, v_neg

In [ ]:
print(len(train_pos), len(train_neg), len(dev_pos),len(dev_neg))

266041 177218 2000 2000


In [ ]:
def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)  # input padding과 attention 마스크 순서 바꿈
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [ ]:
#train_batch_size=32
#eval_batch_size=64
batch_size = 128  # batch size로 통일, 결과가 가장 좋다고함
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [ ]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [ ]:
# random seed
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 데이터 섞기 실험(cross validation)
wandb.init(
    project="Gwamg Yeon", 
    entity='goorm-team3',
    config = {
  "lr": 2e-5,
  "epochs": 5,
  "batch_size" : 128
})
config = wandb.config
model.train()
optimizer = AdamW(model.parameters(), lr=config.lr) 
# train_epoch = 3
lowest_valid_loss = 9999.
loss_sum = 0  # loss 평균값을 구하기위한 변수

    # 학습 시작
for epoch in range(config.epochs):  # train_epoch -> wandb.config.epochs 로 변경
    train_pos, train_neg, dev_pos, dev_neg = switch_data(train_pos, train_neg, dev_pos, dev_neg, config.epochs, epoch-1) # 전 epoch에 스위칭했던 셋을 돌려놓음
    train_pos, train_neg, dev_pos, dev_neg = switch_data(train_pos, train_neg, dev_pos, dev_neg, config.epochs, epoch)
    train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
    dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config.batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
    dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=config.batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)
    with tqdm(train_loader, unit="batch") as tepoch:
        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch+1}") # epoch 수가 헷갈려서 1 더함
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)

            optimizer.zero_grad()

            output = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        position_ids=position_ids,
                        labels=labels)

            loss = output.loss
            loss.backward()
            loss_sum += loss
            optimizer.step()

            tepoch.set_postfix(loss=loss.item())
            if iteration % 10 == 0:
                wandb.log({"loss_avg per 10 iter":loss_sum/10})  # wandb 로그 추가(loss) -> 10회 반복 당 loss평균으로 변경, 값이 너무 튐
                loss_sum = 0
            if iteration != 0 and iteration % int(len(train_loader) / 10) == 0: # 1epoch 당 accuracy 계산 주기 5 -> 10 
                # Evaluate the model five times per epoch
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                desc='Eval',
                                                                                                position=1,
                                                                                                leave=None):
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    token_type_ids=token_type_ids,
                                    position_ids=position_ids,
                                    labels=labels)

                        logits = output.logits
                        loss = output.loss
                        valid_losses.append(loss.item())

                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        predictions += batch_predictions
                        target_labels += batch_labels
                model.train()  # model.eval()로 검증을 마친 후 학습모드로 다시 돌아가지 않음(결과는 차이 없음)  
                acc = compute_acc(predictions, target_labels)
                valid_loss = sum(valid_losses) / len(valid_losses)
                if lowest_valid_loss > valid_loss:
                    print('Acc for model which have lower valid loss: ', acc)
                    wandb.log({"accuracy" : acc})  # wandb 로그 추가(accuracy)
                    torch.save(model.state_dict(), "./pytorch_model_decay.bin")

wandb: Currently logged in as: kompactss (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Eval:  97%|█████████▋| 31/32 [00:03<00:00,  9.13it/s]
                                                     

Acc for model which have lower valid loss:  0.97


Eval:  97%|█████████▋| 31/32 [00:03<00:00,  9.15it/s]
                                                     

Acc for model which have lower valid loss:  0.9755


...

Eval:  97%|█████████▋| 31/32 [00:03<00:00,  8.99it/s]
                                                     

Acc for model which have lower valid loss:  0.991


Eval:  97%|█████████▋| 31/32 [00:03<00:00,  9.11it/s]
                                                     

Acc for model which have lower valid loss:  0.9895


Epoch 5: 100%|██████████| 3463/3463 [17:27<00:00,  3.31batch/s, loss=0.0233]


In [ ]:
import pandas as pd
test_df = pd.read_csv(drive_path + 'test_no_label.csv')

In [ ]:
test_dataset = test_df['Id']

In [ ]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [ ]:
test = make_id_file_test(tokenizer, test_dataset)

In [ ]:
test[:10]

['101 2009 1005 1055 1037 2878 2047 3325 1998 2047 26389 2169 2051 2017 2175 1012 102',
 '101 2061 15640 2013 2019 2214 5440 1012 102',
 '101 2009 2003 1996 2087 14469 7273 1999 1996 3028 1012 102',
 '101 2079 2025 3696 1037 10084 2007 2122 2111 1012 102',
 '101 1045 2001 6091 1998 2016 2081 2033 2514 2061 6625 1998 6160 1012 102',
 '101 1996 2069 2518 2057 2363 2008 2001 2980 2001 1996 4157 1012 102',
 '101 2053 1010 2025 1996 3924 2012 2004 2226 1010 1996 3924 1999 3502 2152 1012 102',
 '101 2027 3288 2009 2041 2392 2005 2017 1998 2024 2200 14044 1012 102',
 '101 4606 1996 12043 2106 1050 1005 1056 2130 2113 2129 2000 2147 1996 3274 1012 102',
 '101 2027 2031 2019 6581 4989 1997 25025 2015 2000 5454 2013 1012 102']

In [ ]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [ ]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [ ]:
def collate_fn_style_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)
    #sorted_indices = np.array([len(input_id) for input_id in input_ids])  # index가 정렬되서, prediction이 dataset과 매칭이 잘못됨
    attention_mask = torch.tensor(
        [[1] * len(input_id) + [0] * (max_len - len(input_id)) for input_id in
         input_ids])
    input_ids = pad_sequence([torch.tensor(input_id) for input_id in input_ids],
                             batch_first=True)
    token_type_ids = torch.tensor([[0] * len(input_id) for input_id in input_ids])
    position_ids = torch.tensor([list(range(len(input_id))) for input_id in input_ids])

    return input_ids, attention_mask, token_type_ids, position_ids

In [ ]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)
        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)
        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions


Test: 100%|██████████| 32/32 [00:01<00:00, 35.05it/s]
                                                     

In [ ]:
test_df['Category'] = predictions

In [ ]:
test_df.to_csv(drive_path + 'submission_{}_{}_switch.csv'.format(config.epochs, config.lr), index=False)